In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso,Ridge
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.layers.advanced_activations import ReLU, PReLU
from keras.optimizers import SGD, Adam
from scipy.stats import mode
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam
from keras.layers.advanced_activations import ReLU, PReLU

In [2]:
train_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_x_xgboost.csv')
train_y = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_y_xgboost.csv')
test_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\test_x_xgboost.csv')

In [3]:
train_x_nn = pd.read_csv('.//dataset//train_x_NN.csv')
train_y_nn = pd.read_csv('.//dataset//train_y_NN.csv')
test_x_nn = pd.read_csv('.//dataset//test_x_NN.csv')

In [20]:
train_x_poutcome = train_x.drop(['poutcome'], axis=1)
train_y_poutcome = train_x['poutcome']

In [21]:
train_x_poutcome

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score,an_income,pre_total_duration
0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1.0,135234,0
1,29,2,2,2,0,43027,0,0,0,22,1,158,2,702,0,2.0,387243,0
2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,2.0,183780,0
3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0.0,1090331,0
4,48,10,1,0,0,42005,1,0,1,3,0,177,1,273,0,3.0,1176140,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,1,1,1,0,26661,1,0,0,27,7,345,4,425,0,1.0,453237,0
27096,35,7,1,1,0,42150,1,0,0,27,7,121,1,719,0,1.0,632250,0
27097,35,7,1,3,0,34531,0,0,0,28,5,177,2,121,0,2.0,517965,0
27098,30,0,2,1,0,99621,1,0,0,27,7,121,1,100,0,2.0,996210,0


In [10]:
#train_y_poutcome.columns = ['poutcome_failure','poutcome_other','poutcome_success',
                          # 'poutcome_unknown']
#train_y_poutcome

,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,1,0,0
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1
...,...,...,...,...
27095,0,0,0,1
27096,0,0,0,1
27097,0,0,0,1
27098,0,0,0,1


In [22]:
test_x_poutcome = test_x.drop(['poutcome'], axis=1)
test_y_poutcome = test_x['poutcome']

In [18]:
#test_y_poutcome.columns = ['poutcome_failure','poutcome_other','poutcome_success',
                           #'poutcome_unknown']

In [23]:
class Model1xgbsoftmax:

  def __init__(self):
    self.model = None
    self.scaler = None

  def fit (self, tr_x, tr_y, va_x, va_y):
    params = {'booster':'gbtree',
          'objective':'multi:softprob','num_class': 4,
          'silent':1,
          'random_state':71,
          'eta': 0.1,}
    num_round = 500
    self.scaler = StandardScaler()
    self.scaler.fit(tr_x)
    tr_x = self.scaler.transform(tr_x)
    va_x = self.scaler.transform(va_x)
    d_train = xgb.DMatrix(tr_x, label = tr_y)
    d_valid = xgb.DMatrix(va_x, label = va_y)
    watchlist = [(d_train, 'train'),(d_valid, 'eval')]
    self.model = xgb.train(params, d_train, num_round, evals = watchlist,early_stopping_rounds=20)

  def predict(self,x):
    x = self.scaler.transform(x)
    data = xgb.DMatrix(x)
    pred = self.model.predict(data)
    return pred

In [24]:
preds = []
preds_test = []
va_idxes = []

model = Model1xgbsoftmax()
    
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for i , (tr_idx, va_idx) in enumerate(kf.split(train_x)):
    tr_x, va_x = train_x_poutcome.iloc[tr_idx], train_x_poutcome.iloc[va_idx]
    tr_y, va_y = train_y_poutcome.iloc[tr_idx], train_y_poutcome.iloc[va_idx]
    model.fit(tr_x, tr_y, va_x, va_y)
    pred = model.predict(va_x)
    preds.append(pred)
    pred_test = model.predict(test_x_poutcome)
    preds_test.append(pred_test)
    va_idxes.append(va_idx)

[07:48:06] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.08177	eval-merror:0.08723
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.07877	eval-merror:0.08635
[2]	train-merror:0.07705	eval-merror:0.08576
[3]	train-merror:0.07680	eval-merror:0.08561
[4]	train-merror:0.07793	eval-merror:0.08797
[5]	train-merror:0.07705	eval-merror:0.08782
[6]	train-merror:0.07749	eval-merror:0.08723
[7]	train-merror:0.07680	eval-merror:0.08768
[8]	train-merror:0.07641	eval-merror:0.08723
[9]	train-merror:0.07621	eval-merror:0.08664
[10]	train-merror:0

[67]	train-merror:0.05678	eval-merror:0.06996
[68]	train-merror:0.05634	eval-merror:0.06952
[69]	train-merror:0.05589	eval-merror:0.06952
[70]	train-merror:0.05574	eval-merror:0.06937
[71]	train-merror:0.05560	eval-merror:0.06908
[72]	train-merror:0.05515	eval-merror:0.06893
[73]	train-merror:0.05446	eval-merror:0.06893
[74]	train-merror:0.05427	eval-merror:0.06922
[75]	train-merror:0.05387	eval-merror:0.06952
[76]	train-merror:0.05392	eval-merror:0.06967
[77]	train-merror:0.05358	eval-merror:0.06952
[78]	train-merror:0.05323	eval-merror:0.06952
[79]	train-merror:0.05294	eval-merror:0.06982
[80]	train-merror:0.05274	eval-merror:0.06996
[81]	train-merror:0.05269	eval-merror:0.06967
[82]	train-merror:0.05235	eval-merror:0.06937
[83]	train-merror:0.05220	eval-merror:0.06922
[84]	train-merror:0.05186	eval-merror:0.06967
[85]	train-merror:0.05181	eval-merror:0.06982
[86]	train-merror:0.05156	eval-merror:0.06996
[87]	train-merror:0.05122	eval-merror:0.06996
[88]	train-merror:0.05107	eval-mer

Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.07916	eval-merror:0.08959
[2]	train-merror:0.07892	eval-merror:0.08915
[3]	train-merror:0.07995	eval-merror:0.09225
[4]	train-merror:0.08034	eval-merror:0.09166
[5]	train-merror:0.07916	eval-merror:0.08900
[6]	train-merror:0.07764	eval-merror:0.08856
[7]	train-merror:0.07680	eval-merror:0.08930
[8]	train-merror:0.07537	eval-merror:0.08797
[9]	train-merror:0.07498	eval-merror:0.08768
[10]	train-merror:0.07474	eval-merror:0.08694
[11]	train-merror:0.07503	eval-merror:0.08664
[12]	train-merror:0.07277	eval-merror:0.08561
[13]	train-merror:0.07134	eval-merror:0.08531
[14]	train-merror:0.07168	eval-merror:0.08561
[15]	train-merror:0.07164	eval-merror:0.08472
[16]	train-merror:0.07095	eval-merror:0.08517
[17]	train-merror:0.07065	eval-merror:0.08428
[18]	train-merror:0.07026	eval-merror:0.08472
[19]	train-merror:0.06977	eval-mer

In [25]:
a = np.concatenate(va_idxes)
a

array([    0,     1,     2, ..., 27085, 27088, 27094])

In [26]:
b = np.concatenate(preds, axis=0)
b

array([[3.9629471e-01, 8.6090460e-02, 3.4807663e-02, 4.8280719e-01],
       [3.6457251e-03, 7.4977725e-04, 5.1852857e-04, 9.9508601e-01],
       [9.6140051e-01, 1.7303070e-02, 6.5467162e-03, 1.4749637e-02],
       ...,
       [5.8822741e-04, 7.7647914e-04, 5.7538888e-05, 9.9857771e-01],
       [3.8381792e-03, 1.0022228e-03, 3.2156237e-04, 9.9483806e-01],
       [5.4631796e-02, 5.6342922e-02, 1.2733337e-02, 8.7629199e-01]],
      dtype=float32)

In [27]:
c = np.mean(preds_test, axis=0)
preds_test

[array([[5.6294245e-01, 1.1112691e-01, 3.1562382e-01, 1.0306842e-02],
        [1.0098533e-02, 7.2126854e-03, 3.8698453e-03, 9.7881889e-01],
        [1.1720702e-03, 1.1556775e-03, 9.0981688e-04, 9.9676239e-01],
        ...,
        [8.5212110e-04, 1.0301487e-03, 1.0127245e-03, 9.9710506e-01],
        [7.2537787e-04, 6.3430070e-04, 6.4283918e-04, 9.9799746e-01],
        [5.8682011e-03, 9.3793037e-04, 7.4281427e-04, 9.9245101e-01]],
       dtype=float32),
 array([[6.0153228e-01, 5.2086141e-02, 3.3607432e-01, 1.0307250e-02],
        [1.1209112e-02, 1.0033572e-02, 1.8496199e-03, 9.7690773e-01],
        [1.2905407e-03, 1.1721194e-03, 6.4460578e-04, 9.9689269e-01],
        ...,
        [1.1370506e-03, 1.2163515e-03, 9.7287056e-04, 9.9667370e-01],
        [8.1405655e-04, 8.6383207e-04, 4.7342645e-04, 9.9784875e-01],
        [4.7584269e-03, 1.3184767e-03, 1.1185477e-03, 9.9280459e-01]],
       dtype=float32),
 array([[6.39144123e-01, 5.35257868e-02, 3.03997010e-01, 3.33311339e-03],
        [2.3

In [28]:
np.set_printoptions(threshold=30000)
order = np.argsort(a)
print(order)

[    0     1     2     3 20325 13550     4 20326 20327     5  6775  6776
 13551     6 20328  6777  6778 13552 13553 20329     7 13554 13555  6779
 20330     8  6780 13556     9    10  6781    11  6782 20331 13557 20332
  6783    12  6784  6785 20333  6786 20334    13    14  6787 13558 13559
 13560 20335 13561 13562  6788  6789    15 13563 20336 20337  6790    16
    17 13564 13565 20338  6791  6792 20339 20340    18 20341    19    20
 13566  6793    21 13567  6794 20342  6795 13568  6796 13569 13570    22
  6797    23    24  6798 20343    25    26    27 20344    28 20345 13571
    29 20346  6799  6800 13572 13573 13574  6801 20347  6802 13575 20348
 20349 13576    30 20350 20351 20352 20353  6803 13577    31  6804 13578
    32    33  6805 13579 13580 20354  6806 13581 20355    34 13582 20356
  6807 20357  6808    35  6809 13583 20358  6810 13584 20359 13585  6811
 20360 20361    36 13586 20362    37  6812 20363  6813  6814 20364    38
 13587    39  6815  6816  6817 13588  6818  6819 20

In [29]:
pred_train = b[order]
pred_train

array([[3.9629471e-01, 8.6090460e-02, 3.4807663e-02, 4.8280719e-01],
       [3.6457251e-03, 7.4977725e-04, 5.1852857e-04, 9.9508601e-01],
       [9.6140051e-01, 1.7303070e-02, 6.5467162e-03, 1.4749637e-02],
       ...,
       [3.3316296e-02, 3.0283809e-02, 2.2742784e-02, 9.1365719e-01],
       [1.1841190e-02, 5.3267721e-03, 7.1393361e-04, 9.8211813e-01],
       [1.2980828e-03, 9.8441413e-04, 6.8669650e-04, 9.9703074e-01]],
      dtype=float32)

In [31]:
(f'logloss: {log_loss(train_y_poutcome, pred_train, eps=1e-7):.4f}')

'logloss: 0.2190'

# ハイパーパラメータチューニングxgboost

In [39]:
class Model1xgb2softmax:

  def __init__(self):
    self.model = None
    self.scaler = None

  def fit (self, tr_x, tr_y, va_x, va_y):
    params = {'booster':'gbtree',
          'objective':'multi:softprob','num_class': 4,
          'silent':1,
          'random_state':71,
          'eta': 0.1,
          'alpha': 0.0016225779581999205,'colsample_bytree': 0.9,
          'gamma': 5.487628781936845e-05,'lambda':3.677748765757218e-05,
         'max_depth': 4,'min_child_weight': 4.265764874751589,
         'subsample': 0.9}
    num_round = 500
    self.scaler = StandardScaler()
    self.scaler.fit(tr_x)
    tr_x = self.scaler.transform(tr_x)
    va_x = self.scaler.transform(va_x)
    d_train = xgb.DMatrix(tr_x, label = tr_y)
    d_valid = xgb.DMatrix(va_x, label = va_y)
    watchlist = [(d_train, 'train'),(d_valid, 'eval')]
    self.model = xgb.train(params, d_train, num_round, evals = watchlist,early_stopping_rounds=20)

  def predict(self,x):
    x = self.scaler.transform(x)
    data = xgb.DMatrix(x)
    pred = self.model.predict(data)
    return pred

In [40]:
preds = []
preds_test = []
va_idxes = []

model = Model1xgb2softmax()
    
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for i , (tr_idx, va_idx) in enumerate(kf.split(train_x)):
    tr_x, va_x = train_x_poutcome.iloc[tr_idx], train_x_poutcome.iloc[va_idx]
    tr_y, va_y = train_y_poutcome.iloc[tr_idx], train_y_poutcome.iloc[va_idx]
    model.fit(tr_x, tr_y, va_x, va_y)
    pred = model.predict(va_x)
    preds.append(pred)
    pred_test = model.predict(test_x_poutcome)
    preds_test.append(pred_test)
    va_idxes.append(va_idx)

[08:22:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.09230	eval-merror:0.09653
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.09314	eval-merror:0.09506
[2]	train-merror:0.09210	eval-merror:0.09506
[3]	train-merror:0.09186	eval-merror:0.09476
[4]	train-merror:0.09127	eval-merror:0.09358
[5]	train-merror:0.09137	eval-merror:0.09388
[6]	train-merror:0.09141	eval-merror:0.09417
[7]	train-merror:0.09038	eval-merror:0.09476
[8]	train-merror:0.09058	eval-merror:0.09476
[9]	train-merror:0.08989	eval-merror:0.09432
[10]	train-merror:0

[41]	train-merror:0.08354	eval-merror:0.08162
[42]	train-merror:0.08344	eval-merror:0.08162
[43]	train-merror:0.08305	eval-merror:0.08059
[44]	train-merror:0.08236	eval-merror:0.07882
[45]	train-merror:0.08207	eval-merror:0.07956
[46]	train-merror:0.08192	eval-merror:0.07941
[47]	train-merror:0.08177	eval-merror:0.07882
[48]	train-merror:0.08153	eval-merror:0.07882
[49]	train-merror:0.08123	eval-merror:0.07882
[50]	train-merror:0.08113	eval-merror:0.07882
[51]	train-merror:0.08098	eval-merror:0.07793
[52]	train-merror:0.08049	eval-merror:0.07779
[53]	train-merror:0.07995	eval-merror:0.07734
[54]	train-merror:0.07966	eval-merror:0.07720
[55]	train-merror:0.07921	eval-merror:0.07720
[56]	train-merror:0.07902	eval-merror:0.07720
[57]	train-merror:0.07887	eval-merror:0.07675
[58]	train-merror:0.07828	eval-merror:0.07675
[59]	train-merror:0.07798	eval-merror:0.07601
[60]	train-merror:0.07774	eval-merror:0.07631
[61]	train-merror:0.07739	eval-merror:0.07616
[62]	train-merror:0.07715	eval-mer

[61]	train-merror:0.07291	eval-merror:0.08413
[62]	train-merror:0.07252	eval-merror:0.08413
[63]	train-merror:0.07223	eval-merror:0.08399
[64]	train-merror:0.07213	eval-merror:0.08384
[65]	train-merror:0.07183	eval-merror:0.08354
[66]	train-merror:0.07154	eval-merror:0.08354
[67]	train-merror:0.07105	eval-merror:0.08384
[68]	train-merror:0.07114	eval-merror:0.08413
[69]	train-merror:0.07105	eval-merror:0.08443
[70]	train-merror:0.07095	eval-merror:0.08443
[71]	train-merror:0.07070	eval-merror:0.08413
[72]	train-merror:0.07046	eval-merror:0.08384
[73]	train-merror:0.07041	eval-merror:0.08339
[74]	train-merror:0.07011	eval-merror:0.08310
[75]	train-merror:0.06982	eval-merror:0.08339
[76]	train-merror:0.06972	eval-merror:0.08295
[77]	train-merror:0.06972	eval-merror:0.08310
[78]	train-merror:0.06962	eval-merror:0.08325
[79]	train-merror:0.06962	eval-merror:0.08310
[80]	train-merror:0.06952	eval-merror:0.08310
[81]	train-merror:0.06927	eval-merror:0.08280
[82]	train-merror:0.06908	eval-mer

[48]	train-merror:0.07818	eval-merror:0.08561
[49]	train-merror:0.07744	eval-merror:0.08590
[50]	train-merror:0.07724	eval-merror:0.08546
[51]	train-merror:0.07690	eval-merror:0.08487
[52]	train-merror:0.07670	eval-merror:0.08487
[53]	train-merror:0.07656	eval-merror:0.08487
[54]	train-merror:0.07621	eval-merror:0.08428
[55]	train-merror:0.07592	eval-merror:0.08443
[56]	train-merror:0.07547	eval-merror:0.08428
[57]	train-merror:0.07537	eval-merror:0.08428
[58]	train-merror:0.07493	eval-merror:0.08428
[59]	train-merror:0.07454	eval-merror:0.08413
[60]	train-merror:0.07424	eval-merror:0.08413
[61]	train-merror:0.07380	eval-merror:0.08384
[62]	train-merror:0.07346	eval-merror:0.08413
[63]	train-merror:0.07351	eval-merror:0.08384
[64]	train-merror:0.07316	eval-merror:0.08339
[65]	train-merror:0.07306	eval-merror:0.08339
[66]	train-merror:0.07277	eval-merror:0.08384
[67]	train-merror:0.07262	eval-merror:0.08384
[68]	train-merror:0.07237	eval-merror:0.08413
[69]	train-merror:0.07233	eval-mer

In [41]:
a = np.concatenate(va_idxes)
a

array([    0,     1,     2,     3,     6,     9,    13,    20,    25,
          28,    29,    31,    37,    43,    44,    54,    59,    60,
          68,    70,    71,    74,    83,    85,    86,    89,    90,
          91,    93,    96,   110,   117,   120,   121,   129,   135,
         146,   149,   155,   157,   171,   172,   174,   175,   177,
         180,   182,   186,   187,   194,   198,   206,   209,   212,
         213,   217,   222,   228,   231,   232,   235,   236,   237,
         239,   243,   244,   247,   251,   257,   259,   260,   267,
         270,   271,   273,   276,   278,   280,   282,   285,   293,
         297,   298,   301,   306,   314,   316,   324,   328,   332,
         334,   336,   339,   344,   347,   348,   350,   351,   352,
         355,   364,   371,   373,   374,   377,   388,   389,   394,
         400,   401,   403,   411,   414,   423,   424,   427,   432,
         434,   436,   438,   440,   444,   446,   447,   448,   451,
         454,   457,

In [42]:
b = np.concatenate(preds, axis=0)
b

array([[4.62953299e-01, 9.28979889e-02, 3.12553979e-02, 4.12893325e-01],
       [4.43729246e-03, 1.13174971e-03, 7.23079836e-04, 9.93707895e-01],
       [9.34246719e-01, 2.38399953e-02, 1.10374605e-02, 3.08758002e-02],
       ...,
       [9.28924012e-04, 9.87333828e-04, 8.12423968e-05, 9.98002470e-01],
       [3.76694393e-03, 9.65717598e-04, 3.24953435e-04, 9.94942367e-01],
       [9.11112875e-02, 7.44490251e-02, 1.85448676e-02, 8.15894842e-01]],
      dtype=float32)

In [43]:
c = np.mean(preds_test, axis=0)
preds_test

[array([[5.7705438e-01, 1.0316708e-01, 3.0778992e-01, 1.1988637e-02],
        [1.4486378e-02, 1.0678815e-02, 4.2916932e-03, 9.7054315e-01],
        [1.4699062e-03, 1.4345555e-03, 8.5774407e-04, 9.9623775e-01],
        ...,
        [1.5974652e-03, 1.9265922e-03, 1.1042217e-03, 9.9537176e-01],
        [1.0230290e-03, 8.6170441e-04, 4.3196059e-04, 9.9768329e-01],
        [9.0479115e-03, 1.3562383e-03, 1.3137318e-03, 9.8828214e-01]],
       dtype=float32),
 array([[6.21161759e-01, 8.58192965e-02, 2.81758070e-01, 1.12608690e-02],
        [1.18401395e-02, 1.03074200e-02, 2.55254190e-03, 9.75299895e-01],
        [1.97008997e-03, 1.75537379e-03, 7.63161981e-04, 9.95511413e-01],
        ...,
        [2.56632152e-03, 2.51813349e-03, 1.28359522e-03, 9.93632019e-01],
        [9.47241497e-04, 7.25640741e-04, 2.87894130e-04, 9.98039305e-01],
        [6.71682926e-03, 1.21413462e-03, 8.23971815e-04, 9.91245091e-01]],
       dtype=float32),
 array([[5.9974462e-01, 7.9898916e-02, 3.1480363e-01, 5.552833

In [44]:
np.set_printoptions(threshold=30000)
order = np.argsort(a)
print(order)

[    0     1     2     3 20325 13550     4 20326 20327     5  6775  6776
 13551     6 20328  6777  6778 13552 13553 20329     7 13554 13555  6779
 20330     8  6780 13556     9    10  6781    11  6782 20331 13557 20332
  6783    12  6784  6785 20333  6786 20334    13    14  6787 13558 13559
 13560 20335 13561 13562  6788  6789    15 13563 20336 20337  6790    16
    17 13564 13565 20338  6791  6792 20339 20340    18 20341    19    20
 13566  6793    21 13567  6794 20342  6795 13568  6796 13569 13570    22
  6797    23    24  6798 20343    25    26    27 20344    28 20345 13571
    29 20346  6799  6800 13572 13573 13574  6801 20347  6802 13575 20348
 20349 13576    30 20350 20351 20352 20353  6803 13577    31  6804 13578
    32    33  6805 13579 13580 20354  6806 13581 20355    34 13582 20356
  6807 20357  6808    35  6809 13583 20358  6810 13584 20359 13585  6811
 20360 20361    36 13586 20362    37  6812 20363  6813  6814 20364    38
 13587    39  6815  6816  6817 13588  6818  6819 20

In [45]:
pred_train = b[order]
pred_train

array([[4.62953299e-01, 9.28979889e-02, 3.12553979e-02, 4.12893325e-01],
       [4.43729246e-03, 1.13174971e-03, 7.23079836e-04, 9.93707895e-01],
       [9.34246719e-01, 2.38399953e-02, 1.10374605e-02, 3.08758002e-02],
       ...,
       [3.69574279e-02, 4.35058735e-02, 1.86077226e-02, 9.00928974e-01],
       [1.19358767e-02, 5.21681085e-03, 4.12509369e-04, 9.82434809e-01],
       [2.27696216e-03, 1.34207460e-03, 7.69517967e-04, 9.95611489e-01]],
      dtype=float32)

In [46]:
(f'logloss: {log_loss(train_y_poutcome, pred_train, eps=1e-7):.4f}')

'logloss: 0.2182'

# ハイパーパラメータチューニング

In [35]:
import xgboost as xgb


#モデルの定義
class Model:
    
    def __init__(self,params=None):
        self.params = None
        if params is None:
            self.params = {}
        else :
            self.params = params
            
    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'booster':'gbtree',
          'objective':'multi:softprob','num_class':4,
          'silent':1,
          'random_state':71,
          'eta': 0.1,}
        params.update(self.params)
        num_round = 500
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchilist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals =watchilist)
        
        
    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [36]:
from hyperopt import hp

from hyperopt import fmin, tpe, hp , STATUS_OK, Trials
from sklearn.metrics import log_loss

def score(params) :
    
    params['max_depth'] = int(params['max_depth'])
    
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss : {score : .4f}')
    
    history.append((params, score))
    
    return {'loss': score, 'status': STATUS_OK}



param_space = {
    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.1), np.log(10)),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),
    'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
    # 余裕があればalpha, lambdaも調整する
    'alpha' : hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),
    'lambda' : hp.loguniform('lambda', np.log(1e-6), np.log(10.0)),
}



max_evals = 10
trials = Trials()
history = []
fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals = max_evals)


history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params : {best[0]}, score:{best[1]:.4f}')

[07:55:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.08177	eval-merror:0.09210                                                                           

[1]	train-merror:0.08241	eval-merror:0.09151                                                                           

[2]	train-merror:0.08034	eval-merror:0.09225                                                                           

[3]	train-merror:0.07587	eval-merror:0.08812                                                                           

[4]	train-merror:0.07651	eval-merror:0.08797                                                                      

[64]	train-merror:0.05230	eval-merror:0.08059                                                                          

[65]	train-merror:0.05181	eval-merror:0.08030                                                                          

[66]	train-merror:0.05156	eval-merror:0.08059                                                                          

[67]	train-merror:0.05112	eval-merror:0.08103                                                                          

[68]	train-merror:0.05092	eval-merror:0.08074                                                                          

[69]	train-merror:0.05038	eval-merror:0.08044                                                                          

[70]	train-merror:0.05004	eval-merror:0.08074                                                                          

[71]	train-merror:0.04984	eval-merror:0.08059                                                                          

[72]	train-merror:0.04959	eval-m

[131]	train-merror:0.03154	eval-merror:0.07970                                                                         

[132]	train-merror:0.03139	eval-merror:0.07941                                                                         

[133]	train-merror:0.03095	eval-merror:0.07985                                                                         

[134]	train-merror:0.03070	eval-merror:0.07941                                                                         

[135]	train-merror:0.03021	eval-merror:0.07985                                                                         

[136]	train-merror:0.02996	eval-merror:0.07985                                                                         

[137]	train-merror:0.02977	eval-merror:0.07926                                                                         

[138]	train-merror:0.02952	eval-merror:0.07941                                                                         

[139]	train-merror:0.02942	eval-

[198]	train-merror:0.01668	eval-merror:0.07956                                                                         

[199]	train-merror:0.01653	eval-merror:0.07985                                                                         

[200]	train-merror:0.01638	eval-merror:0.08000                                                                         

[201]	train-merror:0.01629	eval-merror:0.08000                                                                         

[202]	train-merror:0.01619	eval-merror:0.08000                                                                         

[203]	train-merror:0.01589	eval-merror:0.08030                                                                         

[204]	train-merror:0.01550	eval-merror:0.08015                                                                         

[205]	train-merror:0.01545	eval-merror:0.08030                                                                         

[206]	train-merror:0.01511	eval-

[265]	train-merror:0.00787	eval-merror:0.07985                                                                         

[266]	train-merror:0.00777	eval-merror:0.08000                                                                         

[267]	train-merror:0.00772	eval-merror:0.08000                                                                         

[268]	train-merror:0.00758	eval-merror:0.08015                                                                         

[269]	train-merror:0.00753	eval-merror:0.08000                                                                         

[270]	train-merror:0.00743	eval-merror:0.08000                                                                         

[271]	train-merror:0.00738	eval-merror:0.08000                                                                         

[272]	train-merror:0.00738	eval-merror:0.07985                                                                         

[273]	train-merror:0.00738	eval-

...

[209]	train-merror:0.05151	eval-merror:0.08044                                                                         

[210]	train-merror:0.05156	eval-merror:0.08030                                                                         

[211]	train-merror:0.05132	eval-merror:0.08044                                                                         

[212]	train-merror:0.05117	eval-merror:0.08044                                                                         

[213]	train-merror:0.05132	eval-merror:0.08044                                                                         

[214]	train-merror:0.05112	eval-merror:0.08044                                                                         

[215]	train-merror:0.05097	eval-merror:0.08044                                                                         

[216]	train-merror:0.05092	eval-merror:0.08044                                                                         

[217]	train-merror:0.05082	eval-

[276]	train-merror:0.04443	eval-merror:0.08118                                                                         

[277]	train-merror:0.04433	eval-merror:0.08133                                                                         

[278]	train-merror:0.04448	eval-merror:0.08133                                                                         

[279]	train-merror:0.04428	eval-merror:0.08118                                                                         

[280]	train-merror:0.04418	eval-merror:0.08133                                                                         

[281]	train-merror:0.04418	eval-merror:0.08103                                                                         

[282]	train-merror:0.04423	eval-merror:0.08103                                                                         

[283]	train-merror:0.04394	eval-merror:0.08103                                                                         

[284]	train-merror:0.04379	eval-

[343]	train-merror:0.03838	eval-merror:0.07956                                                                         

[344]	train-merror:0.03823	eval-merror:0.07970                                                                         

[345]	train-merror:0.03813	eval-merror:0.07970                                                                         

[346]	train-merror:0.03813	eval-merror:0.07985                                                                         

[347]	train-merror:0.03793	eval-merror:0.08000                                                                         

[348]	train-merror:0.03788	eval-merror:0.08015                                                                         

[349]	train-merror:0.03788	eval-merror:0.08000                                                                         

[350]	train-merror:0.03793	eval-merror:0.07985                                                                         

[351]	train-merror:0.03793	eval-

[410]	train-merror:0.03365	eval-merror:0.08015                                                                         

[411]	train-merror:0.03375	eval-merror:0.08030                                                                         

[412]	train-merror:0.03370	eval-merror:0.08059                                                                         

[413]	train-merror:0.03341	eval-merror:0.08059                                                                         

[414]	train-merror:0.03336	eval-merror:0.08059                                                                         

[415]	train-merror:0.03331	eval-merror:0.08030                                                                         

[416]	train-merror:0.03326	eval-merror:0.08030                                                                         

[417]	train-merror:0.03306	eval-merror:0.08030                                                                         

[418]	train-merror:0.03296	eval-

[477]	train-merror:0.02858	eval-merror:0.08015                                                                         

[478]	train-merror:0.02839	eval-merror:0.08015                                                                         

[479]	train-merror:0.02839	eval-merror:0.08030                                                                         

[480]	train-merror:0.02834	eval-merror:0.08030                                                                         

[481]	train-merror:0.02834	eval-merror:0.08030                                                                         

[482]	train-merror:0.02834	eval-merror:0.08044                                                                         

[483]	train-merror:0.02814	eval-merror:0.08030                                                                         

[484]	train-merror:0.02809	eval-merror:0.08044                                                                         

[485]	train-merror:0.02819	eval-